# Globus Report (2020 & 2021)

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
from datetime import datetime
from matplotlib import pyplot as plt

## Set Variables

In [ ]:
report_ID_2020 = 2020091
report_ID_2021 = 2021039
today = datetime(year=2020, month=9, day=25)
current_raw_file = 'Reports/globus_raw_data_09-18-20.csv'
previous_report_file = 'Reports/globus_data_09-18-20.csv'
new_report_file = 'Reports/globus_data_09-25-20.csv'

## Build Current Data

In [ ]:
curr_df = pd.read_csv(current_raw_file).drop_duplicates(subset=['DepartureID', 'field']).pivot(index='DepartureID', columns='field', values='value').reset_index()
curr_df = curr_df[['DepartureID', 'DepartureDate', 'ActualPriceUSD', 'OriginalPriceUSD', 'Available', 'Status', 'Notes']]

curr_df['DepartureDate'] = pd.to_datetime(curr_df['DepartureDate'], format='%d-%b-%Y')
curr_df.sort_values(by=['DepartureDate'], ascending=True, inplace=True)
curr_df = curr_df.loc[curr_df['DepartureDate'] < datetime(year=2022, month=1, day=1, hour=0, minute=0)]

report_ID = curr_df['DepartureDate'].apply(lambda x : report_ID_2020 if x < datetime(year=2021, month=1, day=1, hour=0, minute=0) else report_ID_2021)

curr_df.insert(0, 'ReportID', report_ID)
curr_df.insert(5, 'ActualPriceAUD', np.NaN)
curr_df.insert(6, 'OriginalPriceAUD', np.NaN)
curr_df.insert(8, 'Type', '')
curr_df.insert(10, 'Pax', np.NaN)

curr_df['ActualPriceUSD'] = curr_df['ActualPriceUSD'].str.replace('$', '').astype(float)
curr_df['OriginalPriceUSD'] = curr_df['OriginalPriceUSD'].str.replace('$', '').astype(float)

# curr_df.dropna(subset=['Available'], inplace=True)
booleanDictionary = {'True': True, 'False': False}
curr_df['Available'] = curr_df['Available'].replace(booleanDictionary)

curr_df

## Build Previous Data

In [ ]:
prev_df = pd.read_csv(previous_report_file)
prev_df['ReportID'] = prev_df['ReportID'].astype(int)
departure_code = prev_df['DepartureID'].str.split(pat='-', expand=True)[1]
day_numbers = departure_code.str[0:2]
get_char = lambda x : str(ord(x[2]) - 64)
month_numbers = departure_code.apply(get_char)
year_numbers = departure_code.str[3:5]
departure_date = pd.to_datetime(day_numbers + '-' + month_numbers + '-' + year_numbers, format='%d-%m-%y')

prev_df.insert(2, 'DepartureDate', departure_date)
prev_df.sort_values(by=['DepartureDate'], ascending=True, inplace=True)

prev_df['Notes'] = prev_df['Notes'].astype(str)
prev_df['Notes'] = prev_df['Notes'].str.replace('nan', '')

prev_df

## Check Departure/Status Count (2020)

In [ ]:
curr_plot_df = curr_df['DepartureID'].loc[curr_df['DepartureDate'].dt.year == 2020].groupby([curr_df['DepartureDate'].dt.month, curr_df['Status']]).count()
prev_plot_df = prev_df['DepartureID'].loc[prev_df['DepartureDate'].dt.year == 2020].groupby([prev_df['DepartureDate'].dt.month, prev_df['Status']]).count()
plot_df = pd.concat([prev_plot_df, curr_plot_df], axis=1)
plot_df.columns = ['Previous', 'Current']
plot_df = plot_df.unstack(level=-1, fill_value=0)


months = plot_df.index
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


if 'Available' in plot_df['Previous'].columns:
    previous_available_counts = plot_df['Previous']['Available']
    previous_available_counts.loc[previous_available_counts.isna()] = 0
else:
    previous_available_counts = 0

if 'Not Available' in plot_df['Previous'].columns:
    previous_notavail_counts = plot_df['Previous']['Not Available']
    previous_notavail_counts.loc[previous_notavail_counts.isna()] = 0
else:
    previous_notavail_counts = 0

if 'Cancelled / Sold Out' in plot_df['Previous'].columns:
    previous_removed_counts = plot_df['Previous']['Cancelled / Sold Out']
    previous_removed_counts.loc[previous_removed_counts.isna()] = 0
else:
    previous_removed_counts = 0


if 'Available' in plot_df['Current'].columns:
    current_available_counts = plot_df['Current']['Available']
    current_available_counts.loc[current_available_counts.isna()] = 0
else:
    current_available_counts = 0

if 'Not Available' in plot_df['Current'].columns:
    current_notavail_counts = plot_df['Current']['Not Available']
    current_notavail_counts.loc[current_notavail_counts.isna()] = 0
else:
    current_notavail_counts = 0

if 'Cancelled / Sold Out' in plot_df['Current'].columns:
    current_removed_counts = plot_df['Current']['Cancelled / Sold Out']
    current_removed_counts.loc[current_removed_counts.isna()] = 0
else:
    current_removed_counts = 0


plt.style.use('fivethirtyeight')
width = 0.35
fig, ax = plt.subplots()

ax.bar(months - width/2, previous_available_counts, width=width, color='#1666BE', label='Available')
ax.bar(months - width/2, previous_notavail_counts, width=width, color='#A4A49C', bottom=previous_available_counts, label='Not Available')
ax.bar(months - width/2, previous_removed_counts, width=width, color='#AD1E45', bottom=previous_available_counts + previous_notavail_counts, label='Removed from Website')

ax.bar(months + width/2, current_available_counts, width=width, color='#1666BE')
ax.bar(months + width/2, current_notavail_counts, width=width, color='#A4A49C', bottom=current_available_counts)
ax.bar(months - width/2, current_removed_counts, width=width, color='#AD1E45', bottom=current_available_counts + current_notavail_counts)

plt.xticks(ticks=months, labels=month_labels)
plt.legend()
plt.show()

## Check Departure/Status Count (2021)

In [ ]:
curr_plot_df = curr_df['DepartureID'].loc[curr_df['DepartureDate'].dt.year == 2021].groupby([curr_df['DepartureDate'].dt.month, curr_df['Status']]).count()
prev_plot_df = prev_df['DepartureID'].loc[prev_df['DepartureDate'].dt.year == 2021].groupby([prev_df['DepartureDate'].dt.month, prev_df['Status']]).count()
plot_df = pd.concat([prev_plot_df, curr_plot_df], axis=1)
plot_df.columns = ['Previous', 'Current']
plot_df = plot_df.unstack(level=-1, fill_value=0)


months = plot_df.index
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


if 'Available' in plot_df['Previous'].columns:
    previous_available_counts = plot_df['Previous']['Available']
    previous_available_counts.loc[previous_available_counts.isna()] = 0
else:
    previous_available_counts = 0

if 'Not Available' in plot_df['Previous'].columns:
    previous_notavail_counts = plot_df['Previous']['Not Available']
    previous_notavail_counts.loc[previous_notavail_counts.isna()] = 0
else:
    previous_notavail_counts = 0

if 'Cancelled / Sold Out' in plot_df['Previous'].columns:
    previous_removed_counts = plot_df['Previous']['Cancelled / Sold Out']
    previous_removed_counts.loc[previous_removed_counts.isna()] = 0
else:
    previous_removed_counts = 0


if 'Available' in plot_df['Current'].columns:
    current_available_counts = plot_df['Current']['Available']
    current_available_counts.loc[current_available_counts.isna()] = 0
else:
    current_available_counts = 0

if 'Not Available' in plot_df['Current'].columns:
    current_notavail_counts = plot_df['Current']['Not Available']
    current_notavail_counts.loc[current_notavail_counts.isna()] = 0
else:
    current_notavail_counts = 0

if 'Cancelled / Sold Out' in plot_df['Current'].columns:
    current_removed_counts = plot_df['Current']['Cancelled / Sold Out']
    current_removed_counts.loc[current_removed_counts.isna()] = 0
else:
    current_removed_counts = 0


plt.style.use('fivethirtyeight')
width = 0.35
fig, ax = plt.subplots()

ax.bar(months - width/2, previous_available_counts, width=width, color='#1666BE', label='Available')
ax.bar(months - width/2, previous_notavail_counts, width=width, color='#A4A49C', bottom=previous_available_counts, label='Not Available')
ax.bar(months - width/2, previous_removed_counts, width=width, color='#AD1E45', bottom=previous_available_counts + previous_notavail_counts, label='Removed from Website')

ax.bar(months + width/2, current_available_counts, width=width, color='#1666BE')
ax.bar(months + width/2, current_notavail_counts, width=width, color='#A4A49C', bottom=current_available_counts)
ax.bar(months - width/2, current_removed_counts, width=width, color='#AD1E45', bottom=current_available_counts + current_notavail_counts)

plt.xticks(ticks=months, labels=month_labels)
plt.legend()
plt.show()

## Check Departure/Status Changes

In [ ]:
curr_status_df = curr_df[['DepartureID', 'DepartureDate', 'Status']].set_index(['DepartureID', 'DepartureDate'])
prev_status_df = prev_df[['DepartureID', 'DepartureDate', 'Status']].set_index(['DepartureID', 'DepartureDate'])
status_df = pd.concat([prev_status_df, curr_status_df], axis=1).reset_index('DepartureDate').sort_values(by='DepartureDate')
status_df.columns = ['DepartureDate', 'Previous', 'Current']

### New Departures

In [ ]:
new_departures = status_df.loc[status_df['Previous'].isna()]

new_departures

### Export New Departures

In [ ]:
if new_departures.empty:
    print('No new departures...')
else:
    export_departures = new_departures['DepartureDate'].reset_index()
    split_id = export_departures['DepartureID'].str.split("-", expand=True)
    export_departures.insert(1, 'OpCode', split_id[0])
    export_departures.insert(2, 'DepartureCode', split_id[1])
    export_departures.insert(4, 'Season', export_departures['DepartureDate'].dt.year)
    export_departures.set_index('DepartureID', inplace=True)

    export_departures.to_csv('collette_new_departures.csv')

### Removed Departures

In [ ]:
removed_departures = status_df.loc[(status_df['Current'].isna()) & (status_df['DepartureDate'] > today)]

removed_departures

### Status Changes

In [ ]:
changed_departures = status_df.loc[(status_df['Previous'] != status_df['Current']) & (status_df['Current'].notna())]

changed_departures

## Filter Previous-Past Departures

### Departure date < Today

In [ ]:
prev_past_df = prev_df[prev_df['DepartureDate'] <= today].sort_values(by=['DepartureDate'], ascending=True)

prev_past_df

## Filter Previous-Future Departures

### Departure date > Today
#### If previously 'Available' --> Status = 'Cancelled / Sold Out'

In [ ]:
prev_future_df = prev_df.loc[prev_df['DepartureDate'] >= today].sort_values(by=['DepartureDate'], ascending=True)
filt = (prev_future_df['Available'] == True)
prev_future_df.loc[filt, ['Available', 'Status', 'Notes']] = [False, 'Cancelled / Sold Out', 'Removed from website']

prev_future_df

## Combine Current and Previous-Past Data

In [ ]:
new_df = pd.concat([curr_df, prev_past_df]).sort_values(by=['DepartureDate'], ascending=True)

new_df

## Combine Current and Previous-Future Data

In [ ]:
new_df = pd.concat([new_df, prev_future_df]).sort_values(by=['ReportID', 'DepartureID'], ascending=True).drop_duplicates(subset='DepartureID', keep='last').sort_values(by='DepartureDate', ascending=True)

new_df

## Check Mixed Data

In [ ]:
# prof = pandas_profiling.ProfileReport(new_df)
# prof.to_file(output_file='gate1_report.html')

## Export CSV

In [ ]:
new_df.drop(columns='ReportID', inplace=True)

report_ID = new_df['DepartureDate'].apply(lambda x : report_ID_2020 if x < datetime(year=2021, month=1, day=1, hour=0, minute=0) else report_ID_2021)
new_df.insert(0, 'ReportID', report_ID)

new_df.drop(columns='DepartureDate', inplace=True)
new_df.set_index(['ReportID', 'DepartureID'], verify_integrity=True, inplace=True)

new_df.to_csv(new_report_file)